In [16]:
import pandas as pd
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sys, os
import csv

In [17]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from data_selecter import *
from data_cleaner import *
from data_visualizer import *
from outlier_handler import OutlierHandler

In [18]:
db_config = {
    "host": "localhost",
    "database": "my_database",
    "user": "yohannes",
    "password": "yohannes6460",
    "port": 5432
}

try:
    connection = psycopg2.connect(**db_config)
    print("Connected to PostgreSQL database!")
    connection.close()
except Exception as e:
    print(f"Connection failed: {e}")

Connected to PostgreSQL database!


In [19]:

try:
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()
    print("Connected to the database.")

    # SQL query to extract data
    query = "SELECT * FROM public.xdr_data"
    cursor.execute(query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Column names
    column_names = [desc[0] for desc in cursor.description]

    # Write to CSV
    output_file = "../src/data/xdr_data_export.csv"
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  # Write headers
        writer.writerows(rows)  # Write data rows
        print(f"Data has been exported to {output_file}.")

except Exception as error:
    print(f"Error: {error}")
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Database connection closed.")

Connected to the database.
Data has been exported to ../src/data/xdr_data_export.csv.
Database connection closed.


In [20]:
raw_data = pd.read_csv("../src/data/xdr_data_export.csv")
raw_data.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


In [21]:
raw_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [22]:
raw_data.size

8250055

In [23]:
# number of data points
raw_data.shape

(150001, 55)

In [24]:
raw_data.columns.tolist()

['Bearer Id',
 'Start',
 'Start ms',
 'End',
 'End ms',
 'Dur. (ms)',
 'IMSI',
 'MSISDN/Number',
 'IMEI',
 'Last Location Name',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'HTTP DL (Bytes)',
 'HTTP UL (Bytes)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur. (ms).1',
 'Handset Manufacturer',
 'Handset Type',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Socia

In [25]:
raw_data64_cols = raw_data.select_dtypes(include='float64').columns
raw_data[raw_data64_cols] = raw_data[raw_data64_cols].astype('float32')

In [27]:
missingCount = raw_data.isnull().sum()
missingCount

Bearer Id                                      991
Start                                            1
Start ms                                         1
End                                              1
End ms                                           1
Dur. (ms)                                        1
IMSI                                           570
MSISDN/Number                                 1066
IMEI                                           572
Last Location Name                            1153
Avg RTT DL (ms)                              27829
Avg RTT UL (ms)                              27812
Avg Bearer TP DL (kbps)                          1
Avg Bearer TP UL (kbps)                          1
TCP DL Retrans. Vol (Bytes)                  88146
TCP UL Retrans. Vol (Bytes)                  96649
DL TP < 50 Kbps (%)                            754
50 Kbps < DL TP < 250 Kbps (%)                 754
250 Kbps < DL TP < 1 Mbps (%)                  754
DL TP > 1 Mbps (%)             

In [38]:
totalCells = np.prod(raw_data.shape)
totalCells

np.int64(8250055)

In [39]:
totalMissing = missingCount.sum()
totalMissing

np.int64(1031392)

In [40]:
print("The TellCo dataset contains:", round(((totalMissing/totalCells) * 100), 2),"%", "missing values.")


The TellCo dataset contains: 12.5 % missing values.


In [41]:
missing_df = missing_values_table(raw_data)
missing_df

The selected dataframe has 55 columns.
There are 41 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Nb of sec with 37500B < Vol UL,130254,86.84,float32
Nb of sec with 6250B < Vol UL < 37500B,111843,74.56,float32
Nb of sec with 125000B < Vol DL,97538,65.02,float32
TCP UL Retrans. Vol (Bytes),96649,64.43,float32
Nb of sec with 31250B < Vol DL < 125000B,93586,62.39,float32
Nb of sec with 1250B < Vol UL < 6250B,92894,61.93,float32
Nb of sec with 6250B < Vol DL < 31250B,88317,58.88,float32
TCP DL Retrans. Vol (Bytes),88146,58.76,float32
HTTP UL (Bytes),81810,54.54,float32
HTTP DL (Bytes),81474,54.32,float32


In [42]:
columns_to_remove = missing_df[missing_df['% of Total Values'] >= 30.00].index.tolist()
columns_to_remove

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'TCP UL Retrans. Vol (Bytes)',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'TCP DL Retrans. Vol (Bytes)',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

In [44]:
columns_to_remove = [col for col in columns_to_remove if col not in ['TCP UL Retrans. Vol (Bytes)',
    'TCP DL Retrans. Vol (Bytes)']]
columns_to_remove

['Nb of sec with 37500B < Vol UL',
 'Nb of sec with 6250B < Vol UL < 37500B',
 'Nb of sec with 125000B < Vol DL',
 'Nb of sec with 31250B < Vol DL < 125000B',
 'Nb of sec with 1250B < Vol UL < 6250B',
 'Nb of sec with 6250B < Vol DL < 31250B',
 'HTTP UL (Bytes)',
 'HTTP DL (Bytes)']

In [45]:
my_df = raw_data.copy()


In [46]:
# Droping the rest columens
my_df = my_df.drop(columns_to_remove, axis=1)
my_df.shape

(150001, 47)

In [48]:
missing_values_table(my_df)


The selected dataframe has 47 columns.
There are 33 columns that have missing values.


,Missing Values,% of Total Values,Dtype
TCP UL Retrans. Vol (Bytes),96649,64.43,float32
TCP DL Retrans. Vol (Bytes),88146,58.76,float32
Avg RTT DL (ms),27829,18.55,float32
Avg RTT UL (ms),27812,18.54,float32
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float32
Bearer Id,991,0.66,float32
Nb of sec with Vol UL < 1250B,793,0.53,float32
UL TP < 10 Kbps (%),792,0.53,float32
UL TP > 300 Kbps (%),792,0.53,float32


In [50]:
# backward filling
fix_missing_bfill(my_df, 'TCP UL Retrans. Vol (Bytes)')

96649 missing values in the column TCP UL Retrans. Vol (Bytes) have been replaced using the backward fill method.


c:\kifiya_ai\tellco\scripts\data_cleaner.py:93: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='bfill')


0         7230.0
1         7230.0
2         7230.0
3         7230.0
4         7230.0
           ...  
149996       NaN
149997       NaN
149998       NaN
149999       NaN
150000       NaN
Name: TCP UL Retrans. Vol (Bytes), Length: 150001, dtype: float32

In [51]:
fix_missing_bfill(my_df, 'TCP DL Retrans. Vol (Bytes)')

88146 missing values in the column TCP DL Retrans. Vol (Bytes) have been replaced using the backward fill method.


0         19520.0
1         19520.0
2         19520.0
3         19520.0
4         19520.0
           ...   
149996        NaN
149997        NaN
149998        NaN
149999        NaN
150000        NaN
Name: TCP DL Retrans. Vol (Bytes), Length: 150001, dtype: float32

In [52]:
my_df['Avg RTT DL (ms)'].skew(skipna=True)
my_df['Avg RTT UL (ms)'].skew(skipna=True)


np.float32(28.457417)

In [53]:
fix_missing_ffill(my_df, 'Avg RTT DL (ms)')

27829 missing values in the column Avg RTT DL (ms) have been replaced using the forward fill method.


c:\kifiya_ai\tellco\scripts\data_cleaner.py:86: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col] = df[col].fillna(method='ffill')


0         42.0
1         65.0
2         65.0
3         65.0
4         65.0
          ... 
149996    32.0
149997    27.0
149998    43.0
149999    37.0
150000    37.0
Name: Avg RTT DL (ms), Length: 150001, dtype: float32

In [54]:
fix_missing_ffill(my_df, 'Avg RTT UL (ms)')


27812 missing values in the column Avg RTT UL (ms) have been replaced using the forward fill method.


0         5.0
1         5.0
2         5.0
3         5.0
4         5.0
         ... 
149996    0.0
149997    2.0
149998    6.0
149999    5.0
150000    5.0
Name: Avg RTT UL (ms), Length: 150001, dtype: float32

In [56]:
missing_values_table(my_df)


The selected dataframe has 47 columns.
There are 31 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Last Location Name,1153,0.77,object
MSISDN/Number,1066,0.71,float32
Bearer Id,991,0.66,float32
Nb of sec with Vol UL < 1250B,793,0.53,float32
UL TP > 300 Kbps (%),792,0.53,float32
UL TP < 10 Kbps (%),792,0.53,float32
50 Kbps < UL TP < 300 Kbps (%),792,0.53,float32
10 Kbps < UL TP < 50 Kbps (%),792,0.53,float32
Nb of sec with Vol DL < 6250B,755,0.50,float32
250 Kbps < DL TP < 1 Mbps (%),754,0.50,float32


In [57]:
fix_missing_value(my_df, 'Handset Type', 'not_known')
fix_missing_value(my_df, 'Handset Manufacturer', 'not_known')

572 missing values in the column Handset Type have been replaced by not_known.
572 missing values in the column Handset Manufacturer have been replaced by not_known.


0           Samsung
1           Samsung
2           Samsung
3         undefined
4           Samsung
            ...    
149996        Apple
149997        Apple
149998        Apple
149999       Huawei
150000    not_known
Name: Handset Manufacturer, Length: 150001, dtype: object

In [58]:
drop_rows_with_missing_values(my_df)


3114 rows containg missing values were dropped.


In [59]:
my_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 146887 entries, 0 to 149995
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Bearer Id                       146887 non-null  float32
 1   Start                           146887 non-null  object 
 2   Start ms                        146887 non-null  float32
 3   End                             146887 non-null  object 
 4   End ms                          146887 non-null  float32
 5   Dur. (ms)                       146887 non-null  float32
 6   IMSI                            146887 non-null  float32
 7   MSISDN/Number                   146887 non-null  float32
 8   IMEI                            146887 non-null  float32
 9   Last Location Name              146887 non-null  object 
 10  Avg RTT DL (ms)                 146887 non-null  float32
 11  Avg RTT UL (ms)                 146887 non-null  float32
 12  Avg Bearer TP DL (kbp

In [60]:
convert_to_datetime(my_df, ['Start', 'End'])
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146887 entries, 0 to 149995
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       146887 non-null  float32       
 1   Start                           146887 non-null  datetime64[ns]
 2   Start ms                        146887 non-null  float32       
 3   End                             146887 non-null  datetime64[ns]
 4   End ms                          146887 non-null  float32       
 5   Dur. (ms)                       146887 non-null  float32       
 6   IMSI                            146887 non-null  float32       
 7   MSISDN/Number                   146887 non-null  float32       
 8   IMEI                            146887 non-null  float32       
 9   Last Location Name              146887 non-null  object        
 10  Avg RTT DL (ms)                 146887 non-null  float32     

In [61]:
strings = my_df.select_dtypes(include='object').columns.tolist()

convert_to_string(my_df, strings)


In [62]:
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146887 entries, 0 to 149995
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       146887 non-null  float32       
 1   Start                           146887 non-null  datetime64[ns]
 2   Start ms                        146887 non-null  float32       
 3   End                             146887 non-null  datetime64[ns]
 4   End ms                          146887 non-null  float32       
 5   Dur. (ms)                       146887 non-null  float32       
 6   IMSI                            146887 non-null  float32       
 7   MSISDN/Number                   146887 non-null  float32       
 8   IMEI                            146887 non-null  float32       
 9   Last Location Name              146887 non-null  string        
 10  Avg RTT DL (ms)                 146887 non-null  float32     

In [63]:
my_df.dtypes

Bearer Id                                float32
Start                             datetime64[ns]
Start ms                                 float32
End                               datetime64[ns]
End ms                                   float32
Dur. (ms)                                float32
IMSI                                     float32
MSISDN/Number                            float32
IMEI                                     float32
Last Location Name                string[python]
Avg RTT DL (ms)                          float32
Avg RTT UL (ms)                          float32
Avg Bearer TP DL (kbps)                  float32
Avg Bearer TP UL (kbps)                  float32
TCP DL Retrans. Vol (Bytes)              float32
TCP UL Retrans. Vol (Bytes)              float32
DL TP < 50 Kbps (%)                      float32
50 Kbps < DL TP < 250 Kbps (%)           float32
250 Kbps < DL TP < 1 Mbps (%)            float32
DL TP > 1 Mbps (%)                       float32
UL TP < 10 Kbps (%) 

In [64]:
int_columns = ['Bearer Id', 'IMSI', 'MSISDN/Number', 'IMEI',]
convert_to_int(my_df, int_columns)

In [65]:
my_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146887 entries, 0 to 149995
Data columns (total 47 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       146887 non-null  int64         
 1   Start                           146887 non-null  datetime64[ns]
 2   Start ms                        146887 non-null  float32       
 3   End                             146887 non-null  datetime64[ns]
 4   End ms                          146887 non-null  float32       
 5   Dur. (ms)                       146887 non-null  float32       
 6   IMSI                            146887 non-null  int64         
 7   MSISDN/Number                   146887 non-null  int64         
 8   IMEI                            146887 non-null  int64         
 9   Last Location Name              146887 non-null  string        
 10  Avg RTT DL (ms)                 146887 non-null  float32     

In [66]:
drop_duplicates(my_df)

No duplicate rows were found.


In [67]:
selected_columns = my_df[["Dur. (ms)","Dur. (ms).1"]]
multiply_by_factor(selected_columns, ['Dur. (ms).1'], 1/1000)
selected_columns['comparison'] = (selected_columns['Dur. (ms)'] == selected_columns['Dur. (ms).1'].apply(math.floor))
selected_columns

c:\kifiya_ai\tellco\scripts\data_cleaner.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col] * factor
C:\Users\Home\AppData\Local\Temp\ipykernel_35524\2472654371.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['comparison'] = (selected_columns['Dur. (ms)'] == selected_columns['Dur. (ms).1'].apply(math.floor))


,Dur. (ms),Dur. (ms).1,comparison
0,1823652.0,1.823653e+06,False
1,1365104.0,1.365104e+06,True
2,1361762.0,1.361763e+06,True
3,1321509.0,1.321510e+06,True
4,1089009.0,1.089010e+06,True
...,...,...,...
149991,61661.0,6.166173e+04,True
149992,84940.0,8.494061e+04,True
149993,125622.0,1.256224e+05,True
149994,113545.0,1.135452e+05,True


In [68]:
print(all(selected_columns['comparison']))

False


In [69]:
backup_df = my_df.copy()
drop_columns(backup_df, ['Dur. (ms)'])

1 column was dropped.


In [70]:
outlier_handler = OutlierHandler(backup_df)
outlier_handler
columns = backup_df.select_dtypes('float32').columns.tolist()
columns

['Start ms',
 'End ms',
 'Avg RTT DL (ms)',
 'Avg RTT UL (ms)',
 'Avg Bearer TP DL (kbps)',
 'Avg Bearer TP UL (kbps)',
 'TCP DL Retrans. Vol (Bytes)',
 'TCP UL Retrans. Vol (Bytes)',
 'DL TP < 50 Kbps (%)',
 '50 Kbps < DL TP < 250 Kbps (%)',
 '250 Kbps < DL TP < 1 Mbps (%)',
 'DL TP > 1 Mbps (%)',
 'UL TP < 10 Kbps (%)',
 '10 Kbps < UL TP < 50 Kbps (%)',
 '50 Kbps < UL TP < 300 Kbps (%)',
 'UL TP > 300 Kbps (%)',
 'Activity Duration DL (ms)',
 'Activity Duration UL (ms)',
 'Dur. (ms).1',
 'Nb of sec with Vol DL < 6250B',
 'Nb of sec with Vol UL < 1250B',
 'Social Media DL (Bytes)',
 'Social Media UL (Bytes)',
 'Google DL (Bytes)',
 'Google UL (Bytes)',
 'Email DL (Bytes)',
 'Email UL (Bytes)',
 'Youtube DL (Bytes)',
 'Youtube UL (Bytes)',
 'Netflix DL (Bytes)',
 'Netflix UL (Bytes)',
 'Gaming DL (Bytes)',
 'Gaming UL (Bytes)',
 'Other DL (Bytes)',
 'Other UL (Bytes)',
 'Total UL (Bytes)',
 'Total DL (Bytes)']

In [71]:
outlier_handler.getOverview(columns)

Name of columns,DL TP > 1 Mbps (%),10 Kbps < UL TP < 50 Kbps (%),250 Kbps < DL TP < 1 Mbps (%),Activity Duration DL (ms),Activity Duration UL (ms),Nb of sec with Vol UL < 1250B,Nb of sec with Vol DL < 6250B,UL TP < 10 Kbps (%),TCP DL Retrans. Vol (Bytes),Avg Bearer TP UL (kbps),...,Youtube DL (Bytes),Google UL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Gaming DL (Bytes),Netflix UL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total DL (Bytes)
Min,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,...,53.0,3.0,105.0,42.0,2516.0,35.0,59.0,3290.0,148.0,7114041.0
Q1,0.0,0.0,0.0,15418.0,22073.0,107.0,87.0,99.0,24875.0,47.0,...,5833462.0,1024371.5,5516549.0,5776625.5,210381664.0,5480202.0,4132622.0,210155456.0,4144551.0,243073400.0
Median,0.0,0.0,0.0,39726.0,47180.0,217.0,201.0,100.0,392067.0,63.0,...,11616334.0,2054793.0,11013447.0,11642708.0,423492384.0,10999348.0,8294516.0,421633216.0,8265817.0,455963872.0
Q3,0.0,0.0,1.0,697298.5,611803.0,2466.5,2612.5,100.0,3078146.0,1136.0,...,17440635.0,3088071.5,16514278.0,17469652.0,633334016.0,16503222.5,12432390.5,631361056.0,12382039.5,665783072.0
Max,94.0,98.0,76.0,136536464.0,144911296.0,604122.0,604061.0,100.0,4294425600.0,58613.0,...,23259098.0,4121357.0,22011962.0,23259188.0,843441920.0,22011956.0,16558794.0,843442496.0,16558816.0,902969600.0
IQR,0.0,0.0,1.0,681880.5,589730.0,2359.5,2525.5,1.0,3053271.0,1089.0,...,11607173.0,2063700.0,10997729.0,11693026.5,422952352.0,11023020.5,8299768.5,421205600.0,8237488.5,422709672.0
Lower fence,0.0,0.0,-1.5,-1007402.75,-862522.0,-3432.25,-3701.25,97.5,-4555031.5,-1586.5,...,-11577297.5,-2071178.5,-10980044.5,-11762914.25,-424046864.0,-11054328.75,-8317030.75,-421652944.0,-8211681.75,-390991108.0
Upper fence,0.0,0.0,2.5,1720119.25,1496398.0,6005.75,6400.75,101.5,7658052.5,2769.5,...,34851394.5,6183621.5,33010871.5,35009191.75,1267762544.0,33037753.25,24882043.25,1263169456.0,24738272.25,1299847580.0
Skew,5.345557,10.941071,4.503234,5.821286,7.321154,7.467915,9.009274,-8.958673,16.738823,4.473627,...,0.000119,0.002242,-0.001265,-0.002172,-0.003914,-0.000273,-0.002566,0.001832,0.001481,-0.003579
Number_of_outliers,36594,31509,29343,26126,25501,24870,24604,21811,21636,21104,...,0,0,0,0,0,0,0,0,0,0


In [72]:
outlier_handler.replace_outliers_with_fences(columns)
outlier_handler.getOverview(columns)

Name of columns,Start ms,End ms,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),50 Kbps < DL TP < 250 Kbps (%),...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
Min,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,77.5,0.0,...,53.0,105.0,42.0,35.0,2516.0,59.0,3290.0,148.0,9493545.0,7114041.0
Q1,250.0,251.0,31.0,2.0,43.0,47.0,24875.0,3297.0,91.0,0.0,...,5833462.0,5516549.0,5776625.5,5480202.0,210381664.0,4132622.0,210155456.0,4144551.0,33218346.0,243073400.0
Median,499.0,500.0,44.0,5.0,64.0,63.0,392067.0,16402.0,100.0,0.0,...,11616334.0,11013447.0,11642708.0,10999348.0,423492384.0,8294516.0,421633216.0,8265817.0,41143144.0,455963872.0
Q3,749.0,750.0,67.0,13.0,20024.0,1136.0,3078146.0,68727.0,100.0,4.0,...,17440635.0,16514278.0,17469652.0,16503222.5,633334016.0,12432390.5,631361056.0,12382039.5,49034880.0,665783072.0
Max,999.0,999.0,121.0,29.5,49995.5,2769.5,7658052.5,166872.0,100.0,10.0,...,23259098.0,22011962.0,23259188.0,22011956.0,843441920.0,16558794.0,843442496.0,16558816.0,72759681.0,902969600.0
IQR,499.0,499.0,36.0,11.0,19981.0,1089.0,3053271.0,65430.0,9.0,4.0,...,11607173.0,10997729.0,11693026.5,11023020.5,422952352.0,8299768.5,421205600.0,8237488.5,15816534.0,422709672.0
Lower fence,-498.5,-497.5,-23.0,-14.5,-29928.5,-1586.5,-4555031.5,-94848.0,77.5,-6.0,...,-11577297.5,-10980044.5,-11762914.25,-11054328.75,-424046864.0,-8317030.75,-421652944.0,-8211681.75,9493545.0,-390991108.0
Upper fence,1497.5,1498.5,121.0,29.5,49995.5,2769.5,7658052.5,166872.0,113.5,10.0,...,34851394.5,33010871.5,35009191.75,33037753.25,1267762544.0,24882043.25,1263169456.0,24738272.25,72759681.0,1299847580.0
Skew,0.000823,-0.001251,1.069405,1.163946,1.280613,1.230407,1.216008,1.203139,-1.22817,1.315047,...,0.000119,-0.001265,-0.002172,-0.000273,-0.003914,-0.002566,0.001832,0.001481,-0.001978,-0.003579
Number_of_outliers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
backup_df['Social Media Data Volume (Bytes)'] = backup_df['Social Media UL (Bytes)'] + backup_df['Social Media DL (Bytes)']
backup_df['Google Data Volume (Bytes)'] = backup_df['Google UL (Bytes)'] + backup_df['Google DL (Bytes)']
backup_df['Email Data Volume (Bytes)'] = backup_df['Email UL (Bytes)'] + backup_df['Email DL (Bytes)']
backup_df['Youtube Data Volume (Bytes)'] = backup_df['Youtube UL (Bytes)'] + backup_df['Youtube DL (Bytes)']
backup_df['Netflix Data Volume (Bytes)'] = backup_df['Netflix UL (Bytes)'] + backup_df['Netflix DL (Bytes)']
backup_df['Gaming Data Volume (Bytes)'] = backup_df['Gaming UL (Bytes)'] + backup_df['Gaming DL (Bytes)']
backup_df['Other Data Volume (Bytes)'] = backup_df['Other UL (Bytes)'] + backup_df['Other DL (Bytes)']
backup_df['Total Data Volume (Bytes)'] = backup_df['Total UL (Bytes)'] + backup_df['Total DL (Bytes)']
backup_df

,Bearer Id,Start,Start ms,End,End ms,IMSI,MSISDN/Number,IMEI,Last Location Name,Avg RTT DL (ms),...,Total UL (Bytes),Total DL (Bytes),Social Media Data Volume (Bytes),Google Data Volume (Bytes),Email Data Volume (Bytes),Youtube Data Volume (Bytes),Netflix Data Volume (Bytes),Gaming Data Volume (Bytes),Other Data Volume (Bytes),Total Data Volume (Bytes)
0,-9223372036854775808,2019-04-04 12:01:00,770.0,2019-04-25 14:35:00,662.0,208201442131968,33664962560,35521208254464,9.16456699548519E+015,42.0,...,36749740.0,308879648.0,1570185.0,2905912.0,3701304.0,18355943.0,17855187.0,292426454.0,180558841.0,345629388.0
1,-9223372036854775808,2019-04-09 13:04:00,235.0,2019-04-25 08:15:00,606.0,208201911894016,33681854464,35794009980928,L77566A,65.0,...,53800392.0,653384960.0,1933278.0,4414096.0,937385.0,39359124.0,35565544.0,609920789.0,541959369.0,707185352.0
2,-9223372036854775808,2019-04-09 17:42:00,1.0,2019-04-25 11:58:00,652.0,208200318058496,33760626688,35281512169472,D42335A,65.0,...,27883638.0,279807328.0,1726277.0,10229119.0,3363124.0,34425236.0,23751202.0,229980254.0,414908339.0,307690966.0
3,-9223372036854775808,2019-04-10 00:31:00,486.0,2019-04-25 07:36:00,171.0,208201408577536,33750343680,35356611182592,T21824A,65.0,...,43324216.0,846028544.0,657493.0,11811761.0,2070983.0,36534765.0,15092588.0,810387898.0,761837219.0,889352760.0
4,-9223372036854775808,2019-04-12 20:10:00,565.0,2019-04-25 10:40:00,954.0,208201408577536,33699794944,35407009939456,D88865A,65.0,...,38542816.0,569138560.0,912788.0,7748843.0,2110349.0,34222252.0,17539798.0,531237065.0,564619826.0,607681376.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149991,7349883387961671680,2019-04-29 07:28:00,794.0,2019-04-30 00:36:00,523.0,208201710567424,33762146304,35860607139840,D10033B,27.0,...,57231076.0,295424928.0,915749.0,12036660.0,1730617.0,17269104.0,38337516.0,268231446.0,263493174.0,352656004.0
149992,-9223372036854775808,2019-04-29 07:28:00,114.0,2019-04-30 07:04:00,724.0,208201408577536,33761273856,35327007784960,D78058B,37.0,...,45923464.0,70341448.0,1752716.0,7283344.0,954877.0,36814543.0,13742615.0,43581296.0,820617823.0,116264912.0
149993,-9223372036854775808,2019-04-29 07:28:00,79.0,2019-04-30 18:22:00,512.0,208201408577536,33626114048,35735310696448,D73542B,46.0,...,43629120.0,623318336.0,2307959.0,12459876.0,2555578.0,26306532.0,40974658.0,571286820.0,179348644.0,666947456.0
149994,-9223372036854775808,2019-04-29 07:28:00,83.0,2019-04-30 15:01:00,268.0,208200318058496,33645656064,35155408322560,T88383B,46.0,...,59377936.0,862796032.0,892867.0,8922614.0,1565542.0,42219856.0,28354118.0,825524952.0,792559826.0,922173968.0


In [74]:
backup_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146887 entries, 0 to 149995
Data columns (total 54 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   Bearer Id                         146887 non-null  int64         
 1   Start                             146887 non-null  datetime64[ns]
 2   Start ms                          146887 non-null  float64       
 3   End                               146887 non-null  datetime64[ns]
 4   End ms                            146887 non-null  float64       
 5   IMSI                              146887 non-null  int64         
 6   MSISDN/Number                     146887 non-null  int64         
 7   IMEI                              146887 non-null  int64         
 8   Last Location Name                146887 non-null  string        
 9   Avg RTT DL (ms)                   146887 non-null  float64       
 10  Avg RTT UL (ms)                   146

In [75]:
backup_df.to_csv('../src/data/my_clean_data2.csv')